In [11]:
import time
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.learning_curve import learning_curve, validation_curve
from sklearn.metrics import make_scorer
#
import xgboost as xgb
#
from blending import BlendedRegressor
from cleaning import TargetThresholdFilter
#
pd.set_option('display.max_columns', 500)  # force pandas to display all columns for better visual inspection
# plot plots inline
%matplotlib inline  

In [3]:
# trn = pd.read_csv('../data/train.csv')
# trn = pd.read_csv('../data/train_10.csv', index_col=0)  # column #0 in our file is DataFrame index
trn = pd.read_csv('../data/train_1.csv', index_col=0)
# test = pd.read_csv('../data/test.csv')

In [4]:
t = time.time()
trn_withRef = trn[trn['Ref'].notnull()]
# test_withRef = test[test['Ref'].notnull()]
del trn
print 'Time elapsed: {:.4f} secs'.format(time.time()-t)  # toc

Time elapsed: 0.0713 secs


In [5]:
# Combine observations by 'Id', aggregate features
t = time.time()
trn_withRef_comb = trn_withRef.groupby('Id').agg(['mean','std','median','count','min', 'max'])
trn_withRef_comb.columns = ['_'.join(tup) for (i,tup) in enumerate(trn_withRef_comb.columns.values)]
trn_withRef_comb = trn_withRef_comb.drop(['Expected_count', 'Expected_median', 'Expected_std', 'Expected_min','Expected_max'], axis =1)
# test_withRef_comb = test_withRef.groupby('Id').agg(['mean','std','median','count','min', 'max'])
# test_withRef_comb.columns = ['_'.join(tup) for (i,tup) in enumerate(test_withRef_comb.columns.values)]
# test_withRef_comb = test_withRef_comb.drop(['Expected_count', 'Expected_median', 'Expected_std', 'Expected_min','Expected_max'], axis =1)
del trn_withRef
print 'Time elapsed: {:.4f} secs'.format(time.time()-t)  # toc

Time elapsed: 0.5505 secs


# Impute after cv split

In [6]:
X = trn_withRef_comb.loc[:, 'minutes_past_mean':'Kdp_5x5_90th_max']  # NOTE: used range slicing on column
y = np.log10(1+trn_withRef_comb['Expected_mean'])
def MAE_logy(ground_truth, predictions):
    """Custom scoring function for log(y) or log(1+y)
       NOTE: please change this if you use another non-linearity on y
    """
    return np.float64(np.mean(np.abs(10**ground_truth - 10**predictions)))

In [15]:
# preprocessing
imp = Imputer(strategy='median', copy=True)  # Get a imputor with column-mean filling config
ss = StandardScaler(copy=False, with_mean=True, with_std=True)
clf = xgb.sklearn.XGBRegressor(n_estimators=20, nthread=2)
pip1 = Pipeline([('imp',imp), ('ss',ss), ('clf', clf)])  # a Pipeline wrapper to chain'em up

In [16]:
scorer = make_scorer(MAE_logy, greater_is_better=True)  # define scoring metric
scores = cross_val_score(estimator=pip1, X=X, y=y, scoring=scorer, cv=10, n_jobs=1, verbose=1)
print np.mean(scores), np.std(scores)

17.3405146982 2.44880093437


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   44.2s finished
